In [1]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
# use NVIDIA Geforce GTX 1650
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# total parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

In [4]:
# training function
def train_and_validate(num_of_epochs, model, train_loader, val_loader, loss_function, optimizer, train_loss_array, val_loss_array):

    for epoch in range(num_of_epochs):
        model.train(True)
        total_batch_train_loss = 0.0
        for batch_index, batch in enumerate(train_loader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            total_batch_train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        epoch_train_loss = total_batch_train_loss / len(train_loader)

        model.train(False)
        total_batch_val_loss = 0.0
        for batch_index, batch in enumerate(val_loader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            with torch.no_grad():
                output = model(x_batch)
                loss = loss_function(output, y_batch)
                total_batch_val_loss += loss.item()
        epoch_val_loss = total_batch_val_loss / len(val_loader)
        
        train_loss_array.append(epoch_train_loss)
        val_loss_array.append(epoch_val_loss)
        print(f"Epoch no.: {epoch + 1} | Training Loss: {epoch_train_loss} | Validation Loss: {epoch_val_loss}")

In [14]:
def plot_loss_comparison(epochs, array1, array2, label1, label2, plot_title):

    if len(array1) != epochs or len(array2) != epochs:
        raise ValueError("Loss arrays must have the same length as the number of epochs.")
     
    epochs_array = range(1, epochs + 1)
    plt.figure(figsize=(6, 6))
    plt.plot(epochs_array, array1, 'b', label=label1)
    plt.plot(epochs_array, array2, 'r', label=label2)
    plt.title(plot_title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [15]:
def plot_test_predictions(actual_values, lstm_predicted_values, custom_predicted_values):
    
    epochs = range(1, len(actual_values) + 1)

    plt.figure(figsize=(12, 6))
    plt.plot(epochs, actual_values, 'g', label='Actual Values')
    plt.plot(epochs, lstm_predicted_values, 'b', label='LSTM Predicted Values')
    plt.plot(epochs, custom_predicted_values, 'r', label='CustomNet Predicted Values')
    plt.title('Test Predictions Comparison')
    plt.xlabel('Samples')
    plt.ylabel('Values')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [16]:
def compare_mse_loss(actual_values, lstm_predicted_values, custom_predicted_values):
    lstm_mse = mean_squared_error(actual_values, lstm_predicted_values)
    custom_mse = mean_squared_error(actual_values, custom_predicted_values)
    
    print(f'LSTM MSE Loss: {lstm_mse}')
    print(f'Custom Network MSE Loss: {custom_mse}')